<a href="https://www.kaggle.com/code/dakotachang/gmm-dsaproject?scriptVersionId=285809540" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install python_speech_features

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5868 sha256=cb9a07c64c8d59b296a6e94839a903d8c85dd6b995cd9fece0cba0208d9e7942
  Stored in directory: /root/.cache/pip/wheels/37/01/19/e6c69a32684ab7b2e3ea4985a571d810cf055c72600e7f9f17
Successfully built python_speech_features


In [2]:
import os
import random
from pathlib import Path
from collections import defaultdict
import numpy as np
import librosa
import librosa.display
from python_speech_features import mfcc, delta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import pickle

In [3]:
def extract_features(audio_path, n_mfcc=40):
    y, sr = librosa.load(audio_path, sr=None, mono=True)

    # pre-emphasis
    y = np.append(y[0], y[1:] - 0.97 * y[:-1])

    mfcc_feat = mfcc(
        y, sr,
        winlen=0.05, winstep=0.01,
        numcep=n_mfcc, nfilt=64, nfft=4096,
        appendEnergy=True
    )

    if mfcc_feat.shape[0] < 3:
        return None

    mfcc_feat = StandardScaler().fit_transform(mfcc_feat)
    d1 = delta(mfcc_feat, 2)
    d2 = delta(d1, 2)

    return np.hstack([mfcc_feat, d1, d2])

In [4]:
rootdirs = [
    '/kaggle/input/sound-of-114-species-of-birds-till-2022/Voice of Birds/Voice of Birds/Andean Guan_sound',
    '/kaggle/input/sound-of-114-species-of-birds-till-2022/Voice of Birds/Voice of Birds/Andean Tinamou_sound',
    '/kaggle/input/sound-of-114-species-of-birds-till-2022/Voice of Birds/Voice of Birds/Black-billed Brushturkey_sound',
    '/kaggle/input/sound-of-114-species-of-birds-till-2022/Voice of Birds/Voice of Birds/Chaco Chachalaca_sound',
    '/kaggle/input/sound-of-114-species-of-birds-till-2022/Voice of Birds/Voice of Birds/Emu_sound',
    '/kaggle/input/sound-of-114-species-of-birds-till-2022/Voice of Birds/Voice of Birds/North Island Brown Kiwi_sound'
]

all_files = []
for folder in rootdirs:
    bird = Path(folder).name.replace("_sound", "")
    for f in Path(folder).glob("*.mp3"):
        all_files.append((bird, f))

species_dict = defaultdict(list)
for bird, f in all_files:
    species_dict[bird].append(f)

train_pairs, test_pairs = [], []
for bird, files in species_dict.items():
    tr, ts = train_test_split(files, test_size=0.3, random_state=42)
    train_pairs += [(bird, f) for f in tr]
    test_pairs  += [(bird, f) for f in ts]

print(f"Train: {len(train_pairs)} | Test: {len(test_pairs)}")

Train: 101 | Test: 45


In [5]:
def train_gmm(features, name, components=16):
    X = np.vstack(features)
    gmm = GaussianMixture(
        n_components=components,
        covariance_type="diag",
        n_init=4,
        max_iter=300
    ).fit(X)

    with open(f"{name}.gmm", "wb") as f:
        pickle.dump(gmm, f)

    return gmm
    
species_features = defaultdict(list)

for bird, file_path in train_pairs:
    feats = extract_features(str(file_path))
    if feats is not None:
        species_features[bird].append(feats)

models = {}
for bird, feats in species_features.items():
    print(f"Training {bird} ({len(feats)} files)")
    models[bird] = train_gmm(feats, bird)

Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x7370616e at of

Training Andean Guan (21 files)
Training Andean Tinamou (21 files)
Training Black-billed Brushturkey (14 files)
Training Chaco Chachalaca (21 files)
Training Emu (11 files)
Training North Island Brown Kiwi (13 files)


In [6]:
def predict_species(audio_path, models):
    feats = extract_features(str(audio_path))
    scores = {b: m.score(feats) for b, m in models.items()}
    return max(scores, key=scores.get)

y_true, y_pred = [], []

for bird, file_path in test_pairs:
    try:
        pred = predict_species(file_path, models)
        y_true.append(bird)
        y_pred.append(pred)
    except:
        pass

print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))

Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.


Accuracy: 0.8
                          precision    recall  f1-score   support

             Andean Guan       0.88      0.78      0.82         9
          Andean Tinamou       0.67      0.89      0.76         9
Black-billed Brushturkey       0.86      1.00      0.92         6
        Chaco Chachalaca       0.73      0.89      0.80         9
                     Emu       1.00      0.50      0.67         6
 North Island Brown Kiwi       1.00      0.67      0.80         6

                accuracy                           0.80        45
               macro avg       0.85      0.79      0.80        45
            weighted avg       0.83      0.80      0.80        45



In [7]:
MAX_FRAMES_PER_FILE = 150

X_frames, labels, frame_files = [], [], []

for bird, file_path in test_pairs:
    feats = extract_features(str(file_path))
    if feats is None:
        continue

    idx = np.random.choice(len(feats), min(MAX_FRAMES_PER_FILE, len(feats)), replace=False)
    for i in idx:
        X_frames.append(feats[i])
        labels.append(bird)
        frame_files.append(file_path.name)

X_frames = np.nan_to_num(np.asarray(X_frames, dtype=np.float32))

pca = PCA(n_components=20, random_state=42)
X_pca = pca.fit_transform(X_frames)

tsne = TSNE(
    n_components=2,
    perplexity=40,
    init="pca",
    learning_rate="auto",
    random_state=42
)
X_tsne = tsne.fit_transform(X_pca)

df = pd.DataFrame({
    "x": X_tsne[:, 0],
    "y": X_tsne[:, 1],
    "species": labels,
    "file": frame_files
})

Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
Note: Illegal Audio-MPEG-Header 0x7370616e at offset 344524.
Note: Trying to resync...
Note: Hit end of (available) data during resync.


In [8]:
fig = px.scatter(
    df,
    x="x",
    y="y",
    color="species",
    opacity=0.6,
    title="Frame-level MFCC Embedding (PCA → t-SNE) — All Species",
    width=900,
    height=650
)

fig.update_traces(marker=dict(size=4))
fig.update_layout(
    xaxis_title="t-SNE x",
    yaxis_title="t-SNE y"
)

fig.show()

In [9]:
def plot_species_grid(df, species_name):
    files = df[df["species"] == species_name]["file"].unique()
    examples = random.sample(list(files), min(3, len(files)))

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            f"Example 1: {examples[0]}",
            f"Example 2: {examples[1]}",
            f"Example 3: {examples[2]}",
            "All Frames (Highlighted)"
        ]
    )

    for i, fname in enumerate(examples):
        sub = df[df["file"] == fname]
        fig.add_trace(
            go.Scatter(x=sub.x, y=sub.y, mode="markers",
                       marker=dict(size=5, opacity=0.7)),
            row=1 if i < 2 else 2,
            col=1 if i % 2 == 0 else 2
        )

    fig.add_trace(
        go.Scatter(x=df.x, y=df.y, mode="markers",
                   marker=dict(size=4, color="lightgray", opacity=0.25)),
        row=2, col=2
    )

    sub = df[df["species"] == species_name]
    fig.add_trace(
        go.Scatter(x=sub.x, y=sub.y, mode="markers",
                   marker=dict(size=5, color="red", opacity=0.85)),
        row=2, col=2
    )

    fig.update_layout(
        title=f"Frame-level MFCC Clusters — {species_name}",
        width=900,
        height=800,
        showlegend=False
    )
    fig.show()

In [10]:
for species in sorted(df["species"].unique()):
    plot_species_grid(df, species)